In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model

In [2]:
data_aug = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    fill_mode="nearest"
)
train_dataset = data_aug.flow_from_directory(
    "Pneumonia//train/",
    #labels="inferred", #labels for each image is already given as it's file name
#     label_mode="binary", #binary classification
    batch_size=32,
    target_size=(224, 224), #resize of images
    class_mode="binary"
)

Found 5216 images belonging to 2 classes.


In [3]:
test_dataset = data_aug.flow_from_directory(
    "Pneumonia//test/",
#     labels="inferred",
#     label_mode="binary",
    batch_size=32,
    target_size=(224, 224),
    class_mode="binary"
)

Found 624 images belonging to 2 classes.


In [4]:
val_dataset = data_aug.flow_from_directory(
    "Pneumonia//val/",
#     labels="inferred",
#     label_mode="binary",
    batch_size=32,
    target_size=(224, 224),
)

Found 16 images belonging to 2 classes.


In [5]:
resnet_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

In [6]:
for layer in resnet_model.layers:
  layer.trainable = False
custom_head = tf.keras.Sequential([
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model = tf.keras.Sequential([resnet_model, custom_head])

In [7]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


In [8]:
model.fit(train_dataset, epochs=10)

Epoch 1/10


163/163 [==============================] - 673s 4s/step - loss: 0.2323 - accuracy: 0.9062
Epoch 2/10
163/163 [==============================] - 355s 2s/step - loss: 0.1792 - accuracy: 0.9294
Epoch 3/10
163/163 [==============================] - 351s 2s/step - loss: 0.1397 - accuracy: 0.9484
Epoch 4/10
163/163 [==============================] - 379s 2s/step - loss: 0.1266 - accuracy: 0.9528
Epoch 5/10
163/163 [==============================] - 374s 2s/step - loss: 0.1367 - accuracy: 0.9500
Epoch 6/10
163/163 [==============================] - 289s 2s/step - loss: 0.1214 - accuracy: 0.9534
Epoch 7/10
163/163 [==============================] - 300s 2s/step - loss: 0.1122 - accuracy: 0.9549
Epoch 8/10
163/163 [==============================] - 314s 2s/step - loss: 0.1087 - accuracy: 0.9594
Epoch 9/10
163/163 [==============================] - 315s 2s/step - loss: 0.0981 - accuracy: 0.9618
Epoch 10/10
163/163 [==============================] - 319s 2s/step - loss: 0.1109 - accu

In [9]:
test_loss, test_accuracy = model.evaluate(test_dataset)

print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

# Save the model
model.save('CNN_pneumonia_model.keras')

20/20 [==============================] - 38s 2s/step - loss: 0.3562 - accuracy: 0.8590
Test loss: 0.3562133312225342
Test accuracy: 0.8589743375778198


In [10]:
def predict_pneumonia_with_cnn(image_path):

    # Load and preprocess the image
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = tf.expand_dims(image, axis=0)

    # Predict the probability of tuberculosis
    prediction = model.predict(image)

    # Get the probability of tuberculosis
    probability_of_pneumonia = prediction[0][0]
    print(probability_of_pneumonia)
    # Convert the probability to a binary output
    if probability_of_pneumonia > 0.5:
        output = 'Pneumonia'
    else:
        output = 'Normal'
    return output


In [11]:
print(predict_pneumonia_with_cnn('unseen_P.jpeg'))

1/1 [==============================] - 1s 1s/step
0.84424937
Pneumonia
